In [ ]:
# Initial imports.
import pandas as pd
import numpy as np
from numpy import random
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Import RDS password
from config import rds_pwd

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.5 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-20 00:42:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.89MB/s    in 0.2s    

2021-05-20 00:42:42 (5.89 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://wine-final-project.czqkltznl3rl.us-east-2.rds.amazonaws.com/winedb"
#config = {"user":"postgres",
          #"password": "<password>",
          #"driver":"org.postgresql.Driver"}
config = {"user":"wineuser",
          "password": rds_pwd,
          "driver":"org.postgresql.Driver"}

In [ ]:
#select_sql = "(SELECT points, variety, avg_temp, avg_airp, avg_humid, avg_precip FROM machine"
wine_df = spark.read.jdbc(url=jdbc_url, table='weather_mapscaled', properties=config)
#county_df = spark.read.jdbc(url=jdbc_url, table='counties', properties=config)
wine_df.show()

+--------------------+---+----+--------------------+--------------------+--------------------+--------------------+------------+------+------------------+--------------------+
|  winery_search_term|lat| lng|            avg_temp|            avg_airp|           avg_humid|          avg_precip|     country|points|           variety|              winery|
+--------------------+---+----+--------------------+--------------------+--------------------+--------------------+------------+------+------------------+--------------------+
|   LaZarre winery US| 36|-121|279.6850000000000...|1018.230000000000...|73.42000000000000...|0.025000000000000000|          US|    88|        Chardonnay|             LaZarre|
|Negretti winery I...| 45|   8|275.6025000000000...|1016.337500000000...|79.04000000000000...|0.070000000000000000|       Italy|    88|        Chardonnay|            Negretti|
|Pico Maccario win...| 45|   8|280.6950000000000...|1017.880000000000...|70.50250000000000...|0.147500000000000000|     

In [ ]:
allwines = wine_df.select('variety','lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points')
allwines.show()

+------------------+---+----+--------------------+--------------------+--------------------+--------------------+-------+
|           variety|lat| lng|            avg_temp|            avg_airp|           avg_humid|          avg_precip|success|
+------------------+---+----+--------------------+--------------------+--------------------+--------------------+-------+
|        Chardonnay| 36|-121|279.6850000000000...|1018.230000000000...|73.42000000000000...|0.025000000000000000|  false|
|        Chardonnay| 45|   8|275.6025000000000...|1016.337500000000...|79.04000000000000...|0.070000000000000000|  false|
|        Chardonnay| 45|   8|280.6950000000000...|1017.880000000000...|70.50250000000000...|0.147500000000000000|  false|
|        Chardonnay| 43|  12|278.1475000000000...|1014.445000000000...|81.86750000000000...|0.182500000000000000|  false|
|        Chardonnay| 43|  12|278.1475000000000...|1014.445000000000...|81.86750000000000...|0.182500000000000000|  false|
|        Chardonnay| 43|

In [ ]:
mer_df = allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points').filter(allwines.variety == 'Merlot').collect()
#mer_df = wine_df.select(variety='Merlot').collect()
merlot = mer_df.toPandas()
merlot

,lat,lng,avg_temp,avg_airp,avg_humid,avg_precid,success
0,36.0,-87.0,277.1825,1020.1425,77.9250,0.1875,0.0
1,36.0,-87.0,277.1825,1020.1425,77.9250,0.1875,0.0
2,42.0,-123.0,276.0250,1019.8875,85.4525,0.0575,0.0
3,44.0,-123.0,276.0250,1019.8875,85.4525,0.0575,0.0
4,36.0,-84.0,277.3750,1020.0125,79.1975,0.2350,0.0
...,...,...,...,...,...,...,...
2969,41.0,-72.0,272.7475,1016.8375,70.1425,0.1400,0.0
2970,41.0,-72.0,272.7475,1016.8375,70.1425,0.1400,0.0
2971,41.0,-72.0,272.7475,1016.8375,70.1425,0.1400,0.0
2972,35.0,-84.0,280.4425,1019.8050,73.0225,0.2600,0.0


In [ ]:
merlot.dtypes

lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precid    float64
success       float64
dtype: object

In [ ]:
county_weath = spark.read.jdbc(url=jdbc_url, table='uscounty', properties=config)
county_weath.show()

+----+---+---+------------+--------+-------------+------------+--------------------+--------------------+--------------------+--------------------+
| zip|lat|lng|        city|state_id|   state_name| county_name|            avg_temp|            avg_airp|           avg_humid|          avg_precip|
+----+---+---+------------+--------+-------------+------------+--------------------+--------------------+--------------------+--------------------+
|1001| 42|-73|      Agawam|      MA|Massachusetts|     Hampden|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000000000|
|1002| 42|-72|     Amherst|      MA|Massachusetts|   Hampshire|270.8050000000000...|1016.122500000000...|72.21500000000000...|0.125000000000000000|
|1005| 42|-72|       Barre|      MA|Massachusetts|   Worcester|270.7250000000000...|1014.960000000000...|71.75250000000000...|0.135000000000000000|
|1029| 42|-73|   East Otis|      MA|Massachusetts|   Berkshire|270.8050000000000...|1016.122500000000...|72.2150

In [ ]:
bigcounty = county_weath.toPandas()
bigcounty.head(10)

,zip,city,state_id,state_name,county_name,lat,lng,avg_temp,avg_airp,avg_humid,avg_precip
0,1001,Agawam,MA,Massachusetts,Hampden,42.0,-73.0,270.805,1016.1225,72.2150,0.125
1,1002,Amherst,MA,Massachusetts,Hampshire,42.0,-72.0,270.805,1016.1225,72.2150,0.125
2,1005,Barre,MA,Massachusetts,Worcester,42.0,-72.0,270.725,1014.9600,71.7525,0.135
3,1029,East Otis,MA,Massachusetts,Berkshire,42.0,-73.0,270.805,1016.1225,72.2150,0.125
4,1054,Leverett,MA,Massachusetts,Franklin,42.0,-72.0,270.805,1016.1225,72.2150,0.125


In [ ]:
bigcounty.dtypes

zip              int64
city            object
state_id        object
state_name      object
county_name     object
lat            float64
lng            float64
avg_temp       float64
avg_airp       float64
avg_humid      float64
avg_precip     float64
dtype: object

In [ ]:
countydrops = ['zip','city','state_id','state_name','county_name']

In [ ]:
mer_df = (allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points').filter(allwines.variety == 'Merlot').collect()).toPandas()
syr_df = (allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points').filter(allwines.variety == 'Syrah').collect()).toPandas()
savyb_df = (allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points').filter(allwines.variety == 'Sauvignon Blanc').collect()).toPandas()
ries_df = (allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points').filter(allwines.variety == 'Riesling').collect()).toPandas()
cabsav_df = (allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points').filter(allwines.variety == 'Cabernet Sauvignon').collect()).toPandas()
char_df = (allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points').filter(allwines.variety == 'Chardonnay').collect()).toPandas()
pino_df = (allwines.select('lat','lng','avg_temp','avg_airp','avg_humid','avg_precip','points').filter(allwines.variety == 'Pinot Noir').collect()).toPandas()

In [ ]:
mer_df.dtypes

lat           float64
lng           float64
avg_temp      float64
avg_airp      float64
avg_humid     float64
avg_precid    float64
success       float64
dtype: object

In [ ]:
winedropcols = ['points']

In [ ]:
#features and targets
variety = mer_df.copy()
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7728494623655914

In [ ]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
mer_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = syr_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.6588594704684317

In [ ]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
syr_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = savyb_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7593220338983051

In [ ]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
savyb_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = ries_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.6945967110415036

In [ ]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
ries_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = cabsav_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7035860190649115

In [ ]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
cabsav_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = char_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.7079646017699115

In [ ]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
char_preds = rf.predict(Xcount)

In [ ]:
#features and targets
variety = pino_df
y = variety['success']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestClassifier(n_estimators=200, random_state=99)
rf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=99, verbose=0,
                       warm_start=False)

In [ ]:
preds = rf.predict(X_test)
acc_score = accuracy_score(y_test, preds)
acc_score

0.6501107244542866

In [ ]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=['zip','city', 'state_id', 'state_name', 'county_name'])
pino_preds = rf.predict(Xcount)

In [ ]:
counties_preds = bigcounty.copy()
counties_preds['mer_success'] = mer_preds
counties_preds['syr_success'] = syr_preds
counties_preds['savyb_success'] = savyb_preds
counties_preds['ries_success'] = ries_preds
counties_preds['cabsav_success'] = cabsav_preds
counties_preds['char_success'] = char_preds
counties_preds['pinot_success'] = pino_preds

In [ ]:
counties_preds.head()

,zip,city,state_id,state_name,county_name,lat,lng,avg_temp,avg_airp,avg_humid,avg_precip,mer_success,rose_success,syr_success,savyb_success,ries_success,cabsav_success,char_success,pinot_success
0,1001,Agawam,MA,Massachusetts,Hampden,42.0,-73.0,270.805,1016.1225,72.2150,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1002,Amherst,MA,Massachusetts,Hampshire,42.0,-72.0,270.805,1016.1225,72.2150,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1005,Barre,MA,Massachusetts,Worcester,42.0,-72.0,270.725,1014.9600,71.7525,0.135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1029,East Otis,MA,Massachusetts,Berkshire,42.0,-73.0,270.805,1016.1225,72.2150,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1054,Leverett,MA,Massachusetts,Franklin,42.0,-72.0,270.805,1016.1225,72.2150,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
counties_preds.to_csv('county_predictions.csv')
counties_preds.to_json('county_predictions.json')

In [ ]:
spark_counties = spark.createDataFrame(counties_preds)
spark_counties.show()

In [ ]:
#spark_counties.write.jdbc(url=jdbc_url, table='county_predictions', mode=mode, properties=config)

In [ ]:
#sparked_preds = spark.read.jdbc(url=jdbc_url, table='county_predictions', properties=config)

In [ ]:
#sparked_preds.show()

In [ ]:
#winners_df = pd.DataFrame(sparked_preds.select('zip','lat','lng','state_id','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success').collect(), columns=['zip','lat','lng','state','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success'], dtype=float)
#winners_df.describe()

In [ ]:
#winners_df.columns

In [ ]:
#winners_df

In [ ]:
#winners_df['mer_success'].value_counts()